In [21]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.losses import BinaryCrossentropy

In [22]:
G_train_df      = pd.read_csv ('../data/preprocessed_data/G_train.csv')
T_train_df      = pd.read_csv ('../data/preprocessed_data/T_train.csv')
G_T_train_df    = pd.read_csv ('../data/preprocessed_data/balanced_G_T_train.csv')

G_cv_df         = pd.read_csv ('../data/preprocessed_data/G_cv.csv')
T_cv_df         = pd.read_csv ('../data/preprocessed_data/T_cv.csv')
G_T_cv_df       = pd.read_csv ('../data/preprocessed_data/G_T_cv.csv')


In [23]:
random_state = 13
frac = 0.1

G_train_df      = G_train_df.sample(frac = frac, random_state=random_state)
T_train_df      = T_train_df.sample(frac = frac, random_state=random_state)
G_T_train_df    = G_T_train_df.sample(frac = frac, random_state=random_state)

In [24]:
y_train = G_T_train_df.drop(columns = ['group ID','technique ID' ]).values
y_train.dtype

# G_train = sampled_G_train_df.drop(columns = ids)
G_train = G_train_df.drop(columns = 'group ID').values

# T_train = sampled_T_train_df.drop(columns = ids)
T_train = T_train_df.drop(columns = 'technique ID').values

In [25]:
G_train_tf = tf.convert_to_tensor(G_train)
T_train_tf = tf.convert_to_tensor(T_train)
y_train_tf = tf.convert_to_tensor(y_train)
train_dataset = tf.data.Dataset.from_tensor_slices(({'input_Group': G_train_tf, 'input_Technique': T_train_tf}, y_train_tf))

batch_size = 32


train_dataset = train_dataset.shuffle(buffer_size=len(G_train_tf))
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

In [26]:
y_cv = G_T_cv_df.drop(columns = ['group ID','technique ID' ]).values
y_cv.dtype

# G_cv = sampled_G_cv_df.drop(columns = ids)
G_cv = G_cv_df.drop(columns = 'group ID').values

# T_cv = sampled_T_cv_df.drop(columns = ids)
T_cv = T_cv_df.drop(columns = 'technique ID').values

In [27]:
G_cv_tf = tf.convert_to_tensor(G_cv)
T_cv_tf = tf.convert_to_tensor(T_cv)
y_cv_tf = tf.convert_to_tensor(y_cv)
cv_dataset = tf.data.Dataset.from_tensor_slices(({'input_Group': G_cv_tf, 'input_Technique': T_cv_tf}, y_cv_tf))
cv_dataset = cv_dataset.batch(batch_size)

In [34]:
import sys
sys.path.append('../models') 

from model_v0_1 import ContentBasedFiltering

In [39]:
# input shapes config
num_G_features = G_train.shape[1]  # remove Group ID during training
num_T_features = T_train.shape[1]   # remove Movie ID during training

model1 = ContentBasedFiltering (num_G_features=num_G_features, num_T_features=num_T_features)

In [40]:
epochs = 50
history = model1.train (
    train_dataset,
    cv_dataset,
    epochs= epochs
)

Epoch 1/50


403/403 [==============================] - 3s 6ms/step - loss: 2.3463 - val_loss: 0.7708
Epoch 2/50
403/403 [==============================] - 2s 5ms/step - loss: 0.7043 - val_loss: 0.7471
Epoch 3/50
403/403 [==============================] - 2s 5ms/step - loss: 0.6689 - val_loss: 0.6592
Epoch 4/50
403/403 [==============================] - 2s 5ms/step - loss: 0.6632 - val_loss: 0.6474
Epoch 5/50
403/403 [==============================] - 2s 5ms/step - loss: 0.7152 - val_loss: 0.7468
Epoch 6/50
403/403 [==============================] - 2s 5ms/step - loss: 0.6684 - val_loss: 0.6784
Epoch 7/50
403/403 [==============================] - 2s 5ms/step - loss: 0.6677 - val_loss: 0.7114
Epoch 8/50
403/403 [==============================] - 2s 5ms/step - loss: 0.6692 - val_loss: 0.7205
Epoch 9/50
403/403 [==============================] - 2s 5ms/step - loss: 0.6701 - val_loss: 0.6573
Epoch 10/50
403/403 [==============================] - 2s 5ms/step - loss: 0.6692 - val_loss: 0.7043
Epoch 11/5